# Install the [hotsline](https://github.com/AntoineGrimaldi/hotsline) package

In [ ]:
%cd /home/antoine/homhots/
!git clone git@github.com:AntoineGrimaldi/hotsline.git
%cd hotsline/
!python3 -m pip install --user -r requirements.txt
!python3 -m pip install --user -e .
%cd ../aprovhots/tutorials/

In [ ]:
import tonic, os
from hots.utils import get_loader
from hots.network import network

# Make a trial of the package with Poker DVS

In [ ]:
import tonic, torch, os
from hots.timesurface import timesurface
from hots.network import network
from hots.utils import get_loader, make_histogram_classification, HOTS_Dataset

print(f'Number of GPU devices available: {torch.cuda.device_count()}')
for N_gpu in range(torch.cuda.device_count()):
    print(f'GPU {N_gpu+1} named {torch.cuda.get_device_name(N_gpu)}')

In [ ]:
transform = tonic.transforms.NumpyAsType(int)
trainset = tonic.datasets.POKERDVS(save_to='../../Data/', train=True, transform=transform)
testset = tonic.datasets.POKERDVS(save_to='../../Data/', train=False, transform=transform)
trainloader = get_loader(trainset)
testloader = get_loader(testset)
num_sample_train = len(trainloader)
num_sample_test = len(testloader)
print(f'number of samples in the training set: {len(trainloader)}')
print(f'number of samples in the testing set: {len(testloader)}')

## Initialization of the network

In [ ]:
name = 'homeohots'
homeo = True
timestr = '2022-05-18'
dataset_name = 'poker'

Rz = [2, 4]
N_neuronz = [8, 16]
tauz = [1e3, 4e3]

hots = network(name, dataset_name, timestr, trainset.sensor_size, nb_neurons = N_neuronz, tau = tauz, R = Rz, homeo = homeo)

## Unsupervised clustering

In [ ]:
if not os.path.exists('../Records/'):
    os.mkdir('../Records/')
    os.mkdir('../Records/networks/')
    os.mkdir('../Records/LR_results/')
path = '../Records/networks/'+hots.name+'.pkl'
if not os.path.exists(path):
    hots.clustering(trainloader, trainset.ordering, record = True)
hots.plotlayers();
hots.plotlearning();

## Coding of the training and testing set

In [ ]:
hots.coding(trainloader, trainset.ordering, trainset.classes, training=True)
hots.coding(testloader, trainset.ordering, trainset.classes, training=False)

## Classification with histogram (post-hoc)

In [ ]:
jitter = (None, None)

train_path = f'../Records/output/train/{hots.name}_{num_sample_train}_{jitter}/'
test_path = f'../Records/output/test/{hots.name}_{num_sample_test}_{jitter}/'

testset = HOTS_Dataset(test_path, trainset.sensor_size, transform=transform, dtype=trainset.dtype)
trainset = HOTS_Dataset(train_path, trainset.sensor_size, transform=transform, dtype=trainset.dtype)

score = make_histogram_classification(trainset, testset, N_neuronz[-1])
print(f' Accuracy: {score*100}%')